# Solutions: Tutorial 01 - Introduction to Panel Data Structures

**Series**: PanelBox - Fundamentals (Solutions)
**Level**: Beginner
**Tutorial**: 01_introduction_panel_data.ipynb

This notebook contains complete solutions to the exercises in Tutorial 01.

---

## Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# PanelBox library
import sys
sys.path.append('/home/guhaase/projetos/panelbox')
import panelbox as pb
from panelbox.core.panel_data import PanelData

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print(f"PanelBox version: {pb.__version__}")
print("Setup complete!")

## Load Data

In [ ]:
# Load Grunfeld dataset
import os
data_path = '/home/guhaase/projetos/panelbox/examples/datasets/grunfeld.csv'
data = pd.read_csv(data_path)

# Identify entity and time columns
entity_col = 'firm' if 'firm' in data.columns else data.columns[0]
time_col = 'year' if 'year' in data.columns else data.columns[1]

# Create PanelData object
panel = PanelData(data, entity_col=entity_col, time_col=time_col)

print(f"Dataset loaded: {panel.n_entities} entities × {panel.n_periods} periods = {panel.n_obs} observations")

---

## Exercise 1: Load and Inspect Another Variable

**Task**: Analyze the `value` variable instead of `invest`.
1. Calculate entity means for `value`
2. Create a time series plot
3. Compute variance decomposition (within vs between)

In [ ]:
# Step 1: Calculate entity means for 'value'
print("="*60)
print("SOLUTION 1: ANALYZE 'VALUE' VARIABLE")
print("="*60)

var = 'value'
entity_means = panel.dataframe.groupby(entity_col)[var].mean()

print(f"\nEntity means for '{var}':")
print(entity_means.sort_values(ascending=False))

# Display statistics
print(f"\nStatistics:")
print(f"  Mean of means: {entity_means.mean():.2f}")
print(f"  Std of means: {entity_means.std():.2f}")
print(f"  Min: {entity_means.min():.2f}")
print(f"  Max: {entity_means.max():.2f}")

In [ ]:
# Step 2: Create time series plot
fig, ax = plt.subplots(figsize=(12, 6))

for entity in panel.dataframe[entity_col].unique():
    entity_data = panel.dataframe[panel.dataframe[entity_col] == entity]
    ax.plot(entity_data[time_col], entity_data[var], marker='o', label=entity, alpha=0.7)

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Firm Value', fontsize=12, fontweight='bold')
ax.set_title('Firm Value Over Time by Firm', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservation: Firm values show strong heterogeneity and upward trends")

In [ ]:
# Step 3: Variance decomposition
print("\n" + "="*60)
print("VARIANCE DECOMPOSITION FOR 'VALUE'")
print("="*60)

# Overall variance
total_var = panel.dataframe[var].var()

# Between variance
entity_means_series = panel.dataframe.groupby(entity_col)[var].mean()
between_var = entity_means_series.var()

# Within variance
def within_variance(group):
    return ((group - group.mean())**2).mean()

within_var = panel.dataframe.groupby(entity_col)[var].apply(within_variance).mean()

print(f"\nVariable: '{var}'")
print(f"\nTotal variance:   {total_var:,.2f}")
print(f"Between variance: {between_var:,.2f} ({100*between_var/total_var:.1f}%)")
print(f"Within variance:  {within_var:,.2f} ({100*within_var/total_var:.1f}%)")

# Visualization
fig, ax = plt.subplots(figsize=(8, 5))
variances = [total_var, between_var, within_var]
labels = ['Total', 'Between\n(across firms)', 'Within\n(over time)']
colors = ['#3498db', '#e74c3c', '#2ecc71']

ax.bar(labels, variances, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Variance', fontsize=12, fontweight='bold')
ax.set_title('Variance Decomposition of Firm Value', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
if between_var > within_var:
    print("  ✓ Between variance dominates: Firms differ substantially in value")
    print("  ✓ Within variance is lower: Less variation over time within each firm")
else:
    print("  ✓ Within variance dominates: Firm values change substantially over time")
    print("  ✓ Between variance is lower: Firms are more similar to each other")

---

## Exercise 2: Create More Complex Transformations

**Task**: Create the following transformations for the `capital` variable:
1. Log transformation: `log_capital`
2. Growth rate: `capital_growth` (percent change from t-1)
3. 3-year moving average: `capital_ma3`

In [ ]:
print("="*60)
print("SOLUTION 2: COMPLEX TRANSFORMATIONS FOR 'CAPITAL'")
print("="*60)

var = 'capital'

# Transformation 1: Log transformation
panel.dataframe['log_capital'] = np.log(panel.dataframe[var])
print("\n✓ Transformation 1: Log transformation created")
print(f"  Formula: log_capital = log(capital)")
print(f"  Use case: Linearize exponential relationships, interpret as elasticities")

# Transformation 2: Growth rate (percent change)
panel.dataframe['capital_growth'] = panel.dataframe.groupby(entity_col)[var].pct_change() * 100
print("\n✓ Transformation 2: Growth rate created")
print(f"  Formula: capital_growth = (capital_t - capital_t-1) / capital_t-1 * 100")
print(f"  Unit: Percentage change from previous period")

# Transformation 3: 3-year moving average
panel.dataframe['capital_ma3'] = panel.dataframe.groupby(entity_col)[var].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
print("\n✓ Transformation 3: 3-year moving average created")
print(f"  Formula: capital_ma3 = (capital_t + capital_t-1 + capital_t-2) / 3")
print(f"  Use case: Smooth out short-term fluctuations")

# Display example
sample_firm = panel.dataframe[entity_col].iloc[0]
sample_data = panel.dataframe[panel.dataframe[entity_col] == sample_firm][[
    time_col, var, 'log_capital', 'capital_growth', 'capital_ma3'
]].head(10)

print(f"\nExample: Firm '{sample_firm}'")
display(sample_data)

In [ ]:
# Visualize transformations
sample_firm = panel.dataframe[entity_col].iloc[0]
sample_data = panel.dataframe[panel.dataframe[entity_col] == sample_firm]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Original capital
axes[0, 0].plot(sample_data[time_col], sample_data[var], marker='o', color='blue')
axes[0, 0].set_title(f'Original Capital (Firm {sample_firm})', fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Capital')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Log capital
axes[0, 1].plot(sample_data[time_col], sample_data['log_capital'], marker='o', color='green')
axes[0, 1].set_title(f'Log Capital (Firm {sample_firm})', fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('log(Capital)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Growth rate
axes[1, 0].plot(sample_data[time_col], sample_data['capital_growth'], marker='o', color='red')
axes[1, 0].axhline(0, color='black', linestyle='--', linewidth=0.8)
axes[1, 0].set_title(f'Capital Growth Rate (Firm {sample_firm})', fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Growth (%)')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Moving average vs original
axes[1, 1].plot(sample_data[time_col], sample_data[var], marker='o', label='Original', alpha=0.5)
axes[1, 1].plot(sample_data[time_col], sample_data['capital_ma3'], marker='s', label='MA(3)', linewidth=2)
axes[1, 1].set_title(f'Capital vs 3-Year MA (Firm {sample_firm})', fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Capital')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  - Log transformation: Makes exponential growth linear")
print("  - Growth rate: Shows period-to-period changes (positive = increase)")
print("  - Moving average: Smooths out volatility, reveals underlying trend")

---

## Exercise 3: Identify Outliers

**Task**: Find entity-year observations with extreme investment values.
1. Calculate z-scores for `invest` (standardized values)
2. Identify observations with |z-score| > 2.5
3. Create a scatter plot highlighting these outliers

In [ ]:
print("="*60)
print("SOLUTION 3: IDENTIFY OUTLIERS IN 'INVEST'")
print("="*60)

var = 'invest'

# Step 1: Calculate z-scores
mean_val = panel.dataframe[var].mean()
std_val = panel.dataframe[var].std()
panel.dataframe['invest_zscore'] = (panel.dataframe[var] - mean_val) / std_val

print(f"\n✓ Z-scores calculated for '{var}'")
print(f"  Mean: {mean_val:.2f}")
print(f"  Std: {std_val:.2f}")
print(f"  Formula: z = (X - mean) / std")

# Step 2: Identify outliers (|z| > 2.5)
threshold = 2.5
outliers = panel.dataframe[np.abs(panel.dataframe['invest_zscore']) > threshold].copy()

print(f"\n✓ Outliers identified (|z-score| > {threshold})")
print(f"  Number of outliers: {len(outliers)} out of {len(panel.dataframe)} observations")
print(f"  Percentage: {100*len(outliers)/len(panel.dataframe):.2f}%")

if len(outliers) > 0:
    print(f"\nOutlier observations:")
    display(outliers[[entity_col, time_col, var, 'invest_zscore']].sort_values('invest_zscore', ascending=False))
else:
    print("  No outliers found with this threshold")

In [ ]:
# Step 3: Scatter plot with outliers highlighted
fig, ax = plt.subplots(figsize=(12, 6))

# Normal observations
normal = panel.dataframe[np.abs(panel.dataframe['invest_zscore']) <= threshold]
ax.scatter(normal.index, normal[var], alpha=0.5, label='Normal', color='blue', s=30)

# Outliers
if len(outliers) > 0:
    ax.scatter(outliers.index, outliers[var], alpha=0.8, label=f'Outliers (|z| > {threshold})',
               color='red', s=100, marker='X', edgecolors='black', linewidths=1.5)
    
    # Annotate outliers
    for idx, row in outliers.iterrows():
        ax.annotate(f"{row[entity_col]}, {row[time_col]}",
                   xy=(idx, row[var]),
                   xytext=(10, 10),
                   textcoords='offset points',
                   fontsize=8,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ax.axhline(mean_val, color='green', linestyle='--', linewidth=2, label='Mean', alpha=0.7)
ax.axhline(mean_val + threshold*std_val, color='orange', linestyle=':', linewidth=1.5,
           label=f'Mean ± {threshold}σ', alpha=0.7)
ax.axhline(mean_val - threshold*std_val, color='orange', linestyle=':', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Observation Index', fontsize=12, fontweight='bold')
ax.set_ylabel('Investment', fontsize=12, fontweight='bold')
ax.set_title('Investment Values: Outlier Detection', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print(f"  - Red X markers: Observations with extremely high or low investment")
print(f"  - These may represent: (1) measurement errors, (2) special events, or (3) genuine extreme values")
print(f"  - Action: Investigate these observations before modeling")

---

## Exercise 4: Panel Balance Check

**Task**: Simulate an unbalanced panel and validate it.
1. Remove 10% of observations randomly from the Grunfeld dataset
2. Create a new `PanelData` object
3. Check if it's balanced
4. Visualize the coverage heatmap

In [ ]:
print("="*60)
print("SOLUTION 4: UNBALANCED PANEL SIMULATION")
print("="*60)

# Step 1: Remove 10% of observations randomly
np.random.seed(42)  # For reproducibility
unbalanced_data = data.sample(frac=0.9, random_state=42)

print(f"\n✓ Created unbalanced panel")
print(f"  Original observations: {len(data)}")
print(f"  After removing 10%: {len(unbalanced_data)}")
print(f"  Removed: {len(data) - len(unbalanced_data)} observations")

In [ ]:
# Step 2: Create new PanelData object
panel_unbalanced = PanelData(unbalanced_data, entity_col=entity_col, time_col=time_col)

print("\n✓ New PanelData object created")
print(f"  Entities: {panel_unbalanced.n_entities}")
print(f"  Periods: {panel_unbalanced.n_periods}")
print(f"  Observations: {panel_unbalanced.n_obs}")

In [ ]:
# Step 3: Check if balanced
is_balanced = panel_unbalanced.is_balanced()

print("\n" + "="*60)
print("PANEL BALANCE CHECK")
print("="*60)

print(f"\nBalanced: {is_balanced}")

if not is_balanced:
    max_obs = panel_unbalanced.n_entities * panel_unbalanced.n_periods
    missing_obs = max_obs - panel_unbalanced.n_obs
    print("✗ Panel is unbalanced")
    print(f"  Maximum possible: {max_obs} observations")
    print(f"  Actual: {panel_unbalanced.n_obs} observations")
    print(f"  Missing: {missing_obs} entity-period combinations")
    
    # Show entity coverage
    entity_counts = panel_unbalanced.dataframe.groupby(entity_col).size()
    print(f"\n  Observations per entity:")
    print(entity_counts.to_string())
else:
    print("✓ Panel is balanced (unexpected after removing 10%!)")

In [ ]:
# Step 4: Visualize coverage heatmap
pivot_table = panel_unbalanced.dataframe.pivot_table(
    index=entity_col,
    columns=time_col,
    values='invest',
    aggfunc='count'
)

coverage = pivot_table.notna().astype(int)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Original balanced panel
pivot_original = panel.dataframe.pivot_table(
    index=entity_col,
    columns=time_col,
    values='invest',
    aggfunc='count'
)
coverage_original = pivot_original.notna().astype(int)

sns.heatmap(coverage_original, cmap='RdYlGn', cbar_kws={'label': 'Observed'},
            linewidths=0.5, linecolor='gray', vmin=0, vmax=1, ax=axes[0])
axes[0].set_title('Original Balanced Panel', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time Period', fontsize=12)
axes[0].set_ylabel('Entity ID', fontsize=12)

# Unbalanced panel
sns.heatmap(coverage, cmap='RdYlGn', cbar_kws={'label': 'Observed'},
            linewidths=0.5, linecolor='gray', vmin=0, vmax=1, ax=axes[1])
axes[1].set_title('Unbalanced Panel (10% removed)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time Period', fontsize=12)
axes[1].set_ylabel('Entity ID', fontsize=12)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  - Green cells: Observed entity-period combinations")
print("  - Red cells: Missing entity-period combinations")
print("  - Left panel: All cells green (balanced)")
print("  - Right panel: Some red cells (unbalanced)")
print("\nImplications:")
print("  - Unbalanced panels require special handling in estimation")
print("  - Some estimators (e.g., standard FE) work fine with unbalanced panels")
print("  - Check if missingness is random or systematic (selection bias!)")

---

## Summary

In these exercises, you practiced:

✅ **Exercise 1**: Analyzing different variables (value vs invest)
✅ **Exercise 2**: Creating complex transformations (log, growth, moving average)
✅ **Exercise 3**: Identifying outliers using z-scores
✅ **Exercise 4**: Working with unbalanced panels

### Key Skills Acquired

1. **Variance decomposition**: Understanding between vs within variation
2. **Data transformations**: Log, growth rates, moving averages
3. **Outlier detection**: Statistical methods to identify extreme values
4. **Panel balance**: Recognizing and handling missing observations

---

### Next Steps

Continue to **Tutorial 02: Model Specification with Formulas** to learn how to specify econometric models.

---

In [ ]:
print("="*60)
print("SOLUTIONS COMPLETED!")
print("="*60)
print("\nYou've successfully completed all exercises in Tutorial 01.")
print("Next: Tutorial 02 - Model Specification with Formulas")
print("\n✨ Great work! ✨")